In [1]:
import numpy as np
from random import choice, randint


In [44]:
class Color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

class Tile:
    EMPTY = 0
    THICKET = 1
    RIVER = 2
    INVALID = 3
    ROAD = 4
    
    CHAR_DICT = {
        EMPTY:'-',
        THICKET:'|',
        RIVER:'%',
        INVALID:'x',
        ROAD:'O',
    }
    
    COL_DICT = {
        THICKET: Color.GREEN,
        RIVER: Color.BLUE,
        INVALID: Color.RED
    }
    
    SCORE_DICT = {
        THICKET: 2
    }
    
def empty_grid(rows=12, cols=21):
    return np.zeros((rows, cols), dtype=np.int8)
    
def repc(grid):
    for row in grid:
        for col in row:
            if col in Tile.CHAR_DICT and col in Tile.COL_DICT:
                print(Tile.COL_DICT[col] + Tile.CHAR_DICT[col] + Color.END + ' ', end='')
            elif col in Tile.CHAR_DICT:
                print(Tile.CHAR_DICT[col] + ' ', end='')
        print()
    print()
    
def fill(grid, char, x, y, xf, yf):
    for i in range(x, xf + 1):
        for j in range(y, yf + 1):
            if grid[i][j] == Tile.EMPTY:
                grid[i][j] = char
    
def from_string(grid, char, x, y, string):
    # returns a list of Roads
    for row, col in string_gen(grid, x, y, string):
        grid[row][col] = char
        

def string_gen(grid, x, y, string):
    row, col = x, y
    yield row, col

    for ch in string:
        if ch == 'u':
            row -= 1
        elif ch == 'd':
            row += 1
        elif ch == 'l':
            col -= 1
        elif ch == 'r':
            col += 1
        else:
            raise ValueError('Invalid Character, use u,d,l, or r')
        yield row, col

def adjacent(grid, x, y, include_diagonals=False):
    dxdy = [(0,-1), (0,1), (-1,0), (1,0)]
    
    if include_diagonals:
        dxdy += [(-1,-1), (-1,1), (1,-1), (1,1)]
        
    for dx, dy in dxdy:
        i = x+dx
        j = y+dy
        if 0<=i<grid.shape[0] and 0<=j<grid.shape[1]:
            yield i,j


def all_coords(grid):
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            yield i,j
            
def mark_invalid(grid):
    for i,j in all_coords(grid):
        if grid[i][j] == Tile.ROAD:
            for x,y in adjacent(grid,i,j):
                if grid[x][y] == Tile.EMPTY:
                    grid[x][y] = Tile.INVALID

def just_road_fixture():
    grid = empty_grid()
    from_string(grid, Tile.ROAD, 2, 7, 'rrrrrdrdrdrddrddllluulllulllluuur')
    mark_invalid(grid)
    return grid

def best_river_fixture():
    grid = just_road_fixture()
    from_string(grid, Tile.RIVER, 4, 0, 'drdrddldlddrrururuuuululululuurrdrdrd')
    fill(grid, Tile.THICKET, 0, 0, 11, 4)
    return grid


def tile_score(grid, x, y):
    score = 0
    if grid[x][y] in Tile.SCORE_DICT:
        score = Tile.SCORE_DICT[grid[x][y]]

    num_adj = num_adjacent(grid, x, y, Tile.RIVER)
    if num_adj > 0:
        return num_adj * 2 * score
    else:
        return score
    
    
def score(grid):
    score = 0
    for i,j in all_coords(grid):
        score += tile_score(grid, i, j)
    return score


def score2(grid):
    """like score but pretends there are thickets"""
    score = 0
    unique_coords = {}
    for i,j in all_coords(grid):
        if grid[i][j] == Tile.RIVER:
            for x,y in adjacent(grid,i,j):
                if grid[x][y] == Tile.EMPTY and (x,y) not in unique_coords:
                    unique_coords[x,y] = True
    
    for x,y in unique_coords:
        num_adj = num_adjacent(grid, x, y, Tile.RIVER)
        if num_adj > 0:
            score += num_adj * 2 * 2
        else:
            score += 2
            
    return score

    
def num_adjacent(grid, x, y, num, include_diagonals=False):
    num_adj = 0
    for i, j in adjacent(grid, x, y, include_diagonals):
        if grid[i][j] == num:
            num_adj += 1

    return num_adj


def check_udlr(grid, x, y):
    """looks for empty adjacent tiles"""
    options = []
    
    dirs = {
        'u':(-1,0),
        'd':(1,0),
        'l':(0,-1),
        'r':(0,1),
    }
    
    for d in dirs:
        i = x+dirs[d][0]
        j = y+dirs[d][1]
        if 0 <= i < grid.shape[0] and 0 <= j < grid.shape[1]:
            if grid[i][j] == Tile.EMPTY:
                options.append(d)
            
    return options

def replace_surrounding(grid, x, y, target, replace):
    for i,j in adjacent(grid, x, y):
        if grid[i][j] == target:
            grid[i][j] = replace
    

def check_coords(grid, x, y):
    """same as check_udlr without the letters"""
    options = []
    
    for i,j in adjacent(grid, x, y):
        if grid[i][j] == Tile.EMPTY or grid[i][j] == Tile.THICKET:
            options.append((i,j))
            
    return options

def random_river_location(rows=12, cols=21):
    starting_locations = []
    for i in range(rows):
        starting_locations.append((i, 0))
        starting_locations.append((i, cols - 1))

    for j in range(cols):
        starting_locations.append((0, j))
        starting_locations.append((rows - 1, j))

    return choice(starting_locations)

def pick(grid, x, y, target=Tile.RIVER, replace=Tile.EMPTY, surround=Tile.THICKET):
    grid[x][y] = target
    for i,j in adjacent(grid, x, y):
        if grid[i][j] == replace:
            grid[i][j] = surround
            
def unpick(grid, x, y):
    pick(grid, x, y, target=Tile.EMPTY, replace=Tile.RIVER, surround=Tile.THICKET)

In [45]:
grid = best_river_fixture()
repc(grid)

% % % | | - - - - - - - - - - - - - - - - 
% | % % | - - x x x x x x - - - - - - - - 
% % | % % - x O O O O O O x - - - - - - - 
| % % | % x O O x x x x O O x - - - - - - 
% | % % | x O x - - - - x O O x - - - - - 
% % | % % x O x x x x - - x O O x - - - - 
| % % | % x O O O O O x x x x O x - - - - 
| | % | % - x x x x O O O O x O O x - - - 
| % % | % - - - - - x x x O x x O x - - - 
% % | % % - - - - - - - x O O O O x - - - 
% | % % | - - - - - - - - x x x x - - - - 
% % % | | - - - - - - - - - - - - - - - - 



In [46]:
score(grid)

234

In [47]:
for i,j in all_coords(grid):
    if grid[i][j] == Tile.THICKET:
        grid[i][j] = Tile.EMPTY

In [48]:
repc(grid)

% % % - - - - - - - - - - - - - - - - - - 
% - % % - - - x x x x x x - - - - - - - - 
% % - % % - x O O O O O O x - - - - - - - 
- % % - % x O O x x x x O O x - - - - - - 
% - % % - x O x - - - - x O O x - - - - - 
% % - % % x O x x x x - - x O O x - - - - 
- % % - % x O O O O O x x x x O x - - - - 
- - % - % - x x x x O O O O x O O x - - - 
- % % - % - - - - - x x x O x x O x - - - 
% % - % % - - - - - - - x O O O O x - - - 
% - % % - - - - - - - - - x x x x - - - - 
% % % - - - - - - - - - - - - - - - - - - 



In [49]:
score2(grid)

244

In [50]:
num_adjacent(grid,10,10,Tile.ROAD)

0

In [52]:
class DFS:
    score = 0
    coords = []
    
    @staticmethod
    def reset():
        DFS.score = 0
        DFS.coords = []

def dfs(grid, last_i, last_j):
    options = check_coords(grid, last_i, last_j)
    while len(options) > 0:
        i,j = options.pop(randint(0, len(options)-1))
        # pick(grid, i, j)
        grid[i][j] = Tile.RIVER
        total = score2(grid)
        if total > DFS.score:
            DFS.score = total
            repc(grid)
            print(total)
        dfs(grid, i, j)
        grid[i][j] = Tile.EMPTY
        # unpick(grid, i, j)
        # replace_surrounding(grid, last_i, last_j, Tile.EMPTY, Tile.THICKET)
        



In [53]:
grid = just_road_fixture()
# pick(grid, 5, 0)
grid[5][0] = Tile.RIVER
repc(grid)
DFS.reset()

- - - - - - - - - - - - - - - - - - - - - 
- - - - - - - x x x x x x - - - - - - - - 
- - - - - - x O O O O O O x - - - - - - - 
- - - - - x O O x x x x O O x - - - - - - 
- - - - - x O x - - - - x O O x - - - - - 
% - - - - x O x x x x - - x O O x - - - - 
- - - - - x O O O O O x x x x O x - - - - 
- - - - - - x x x x O O O O x O O x - - - 
- - - - - - - - - - x x x O x x O x - - - 
- - - - - - - - - - - - x O O O O x - - - 
- - - - - - - - - - - - - x x x x - - - - 
- - - - - - - - - - - - - - - - - - - - - 



In [54]:
grid[4][0]

0

In [55]:
grid = empty_grid(12,5)
# pick(grid, 5, 0)
grid[5][0] = Tile.RIVER
repc(grid)
DFS.reset()
dfs(grid,5,0)

- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
% - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 

- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
% % - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 

20
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- % - - - 
% % - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 

28
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- % % - - 
% % - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 

36
- - - - - 
- - - - - 
- - - - - 
- - % - - 
- % % - - 
% % - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 

44
- - - - - 
- - - - - 
- - - - - 
% % % - - 
- % % - - 
% % - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 

48
- - - - - 
- - - - - 
% - - - - 
% % % - - 
- % % - - 
% % - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 

52
- - - - - 
- % - - - 
% % - - - 
% % % - - 
- % % -

- % % % % 
- % % % % 
% % - % % 
% % % % % 
- % % % % 
% % - % % 
- - - % % 
% % % % - 
% % - - % 
% % - % % 
% - % % - 
% % % - - 

152
- % % % % 
- % % % % 
% % - % % 
% % % % % 
- % % % % 
% % - % % 
- - - % % 
% % % % - 
% % - - % 
- % - % % 
- % - % - 
- % % % - 

156
- % % % % 
- % % % % 
% % - % % 
% % % % % 
- % % % % 
% % - % % 
- - - % % 
% % % % - 
% - % % % 
% - % - % 
% % - % % 
- % % % - 

160


KeyboardInterrupt: 

In [56]:
check_coords(grid,0,4)

[]

In [57]:
grid = just_road_fixture()
# pick(grid, 5, 0)
grid[5][0] = Tile.RIVER
repc(grid)
DFS.reset()
dfs(grid, 5,0)

- - - - - - - - - - - - - - - - - - - - - 
- - - - - - - x x x x x x - - - - - - - - 
- - - - - - x O O O O O O x - - - - - - - 
- - - - - x O O x x x x O O x - - - - - - 
- - - - - x O x - - - - x O O x - - - - - 
% - - - - x O x x x x - - x O O x - - - - 
- - - - - x O O O O O x x x x O x - - - - 
- - - - - - x x x x O O O O x O O x - - - 
- - - - - - - - - - x x x O x x O x - - - 
- - - - - - - - - - - - x O O O O x - - - 
- - - - - - - - - - - - - x x x x - - - - 
- - - - - - - - - - - - - - - - - - - - - 

- - - - - - - - - - - - - - - - - - - - - 
- - - - - - - x x x x x x - - - - - - - - 
- - - - - - x O O O O O O x - - - - - - - 
- - - - - x O O x x x x O O x - - - - - - 
% - - - - x O x - - - - x O O x - - - - - 
% - - - - x O x x x x - - x O O x - - - - 
- - - - - x O O O O O x x x x O x - - - - 
- - - - - - x x x x O O O O x O O x - - - 
- - - - - - - - - - x x x O x x O x - - - 
- - - - - - - - - - - - x O O O O x - - - 
- - - - - - - - - - - - - x x x x - - - - 
- - - - - 

- - - - - - x x x x O O O O x O O x - - - 
- - - - - - - - - - x x x O x x O x - - - 
- - - - - - - - - - - - x O O O O x - - - 
- - - - - - - - - - - - - x x x x - - - - 
- - - - - - - - - - - - - - - - - - - - - 

80
- % % - - % % % % % % % % % - - - - - - - 
- % % % % % - x x x x x x - - - - - - - - 
% % % % - - x O O O O O O x - - - - - - - 
% - - - - x O O x x x x O O x - - - - - - 
% - - - - x O x - - - - x O O x - - - - - 
% - - - - x O x x x x - - x O O x - - - - 
- - - - - x O O O O O x x x x O x - - - - 
- - - - - - x x x x O O O O x O O x - - - 
- - - - - - - - - - x x x O x x O x - - - 
- - - - - - - - - - - - x O O O O x - - - 
- - - - - - - - - - - - - x x x x - - - - 
- - - - - - - - - - - - - - - - - - - - - 

84
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - - - - - - - - 
% % % % - - x O O O O O O x - - - - - - - 
% - - - - x O O x x x x O O x - - - - - - 
% - - - - x O x - - - - x O O x - - - - - 
% - - - - x O x x x x - - x O O x - - - - 
- -

- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - - - 
% - - - - x O x x x x - - x O O x % % - - 
- - - - - x O O O O O x x x x O x - % % - 
- - - - - - x x x x O O O O x O O x - - - 
- - - - - - - - - - x x x O x x O x - - - 
- - - - - - - - - - - - x O O O O x - - - 
- - - - - - - - - - - - - x x x x - - - - 
- - - - - - - - - - - - - - - - - - - - - 

164
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % - 
% - - - - x O x x x x - - x O O x % % % - 
- - - - - x O O O O O x x x x O x - % % - 
- - - - - - x x x x O O O O x O O x - - - 
- - - - - - - - - - x x x O x x O x - - - 
- - - - - - - - - - - - x O O O O x - - - 
- - - - - - - - - - - - - x x x x - - - - 
- - - - - - - - - - - - - - - - - - - - - 

172
-

- - - - - - - - - - - - x O O O O x % % - 
- - - - - - - - - - - - - x x x x - - % % 
- - - - - - - - - - - - % % % % % % % % % 

208
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % % 
% - - - - x O x x x x - - x O O x % % % % 
- - - - - x O O O O O x x x x O x - % % % 
- - - - - - x x x x O O O O x O O x - % % 
- - - - - - - - - - x x x O x x O x % % - 
- - - - - - - - - - - - x O O O O x % % - 
- - - - - - - - - - - % % x x x x - - % % 
- - - - - - - - - - - - % % % % % % % % % 

216
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % % 
% - - - - x O x x x x - - x O O x % % % % 
- - - - - x O O O O O x x x x O x - % % % 
- - - - - - x x x x O O O O x O O x - % % 
-

% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % % 
% - - - - x O x x x x - - x O O x % % % % 
- - - - - x O O O O O x x x x O x - % % % 
- - - - - - x x x x O O O O x O O x - % % 
- - - - - - - - % % x x x O x x O x % % - 
- - - - - - - - % % % - x O O O O x % % - 
- - - - - - - - % - % % % x x x x - - % % 
- - - - - - - - - - - - % % % % % % % % % 

248
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % % 
% - - - - x O x x x x - - x O O x % % % % 
- - - - - x O O O O O x x x x O x - % % % 
- - - - - - x x x x O O O O x O O x - % % 
- - - - - - - - % % x x x O x x O x % % - 
- - - - - - - - % % % - x O O O O x % % - 
- - - - - - - - % - % % % x x x x - - % % 
- - - - - - - - % - - - % % % % % % % % % 

252
- % % - - % % % % % % % % % % - - - - - - 
-

% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % % 
% - - - - x O x x x x - - x O O x % % % % 
- - - - - x O O O O O x x x x O x - % % % 
- - - - - - x x x x O O O O x O O x - % % 
- - - - - - - - % % x x x O x x O x % % - 
- % % % - - - - % % % - x O O O O x % % - 
- - - % - - - - % - % % % x x x x - - % % 
- - - % % % % % % - - - % % % % % % % % % 

304
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % % 
% - - - - x O x x x x - - x O O x % % % % 
- - - - - x O O O O O x x x x O x - % % % 
- - - - - - x x x x O O O O x O O x - % % 
- - - - - - - - % % x x x O x x O x % % - 
% % % % - - - - % % % - x O O O O x % % - 
- - - % - - - - % - % % % x x x x - - % % 
- - - % % % % % % - - - % % % % % % % % % 

308
- % % - - % % % % % % % % % % - - - - - - 
-

332
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % % 
% - - - - x O x x x x - - x O O x % % % % 
- - % - - x O O O O O x x x x O x - % % % 
% % % - - - x x x x O O O O x O O x - % % 
% - - - - - - - % % x x x O x x O x % % - 
% % % % - - - - % % % - x O O O O x % % - 
- - - % - - - - % - % % % x x x x - - % % 
- - - % % % % % % - - - % % % % % % % % % 

340
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % % 
% - - - - x O x x x x - - x O O x % % % % 
- - % % - x O O O O O x x x x O x - % % % 
% % % - - - x x x x O O O O x O O x - % % 
% - - - - - - - % % x x x O x x O x % % - 
% % % % - - - - % % % - x O O O O x % % - 
- - - % - - - - % - % % % x x x x - - % % 
- 

% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % % 
% - - - - x O x x x x - - x O O x % % % % 
- - % % % x O O O O O x x x x O x - % % % 
% % % - % % x x x x O O O O x O O x - % % 
% - - - - % - - % % x x x O x x O x % % - 
% % % % - % - - % % % - x O O O O x % % - 
- - - % - - - - % - % % % x x x x - - % % 
- - - % % % % % % - - - % % % % % % % % % 

376
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - - - x O x - - - - x O O x % % - % % 
% - - - - x O x x x x - - x O O x % % % % 
- - % % % x O O O O O x x x x O x - % % % 
% % % - % % x x x x O O O O x O O x - % % 
% - - - - % - - % % x x x O x x O x % % - 
% % % % - % % - % % % - x O O O O x % % - 
- - - % - - - - % - % % % x x x x - - % % 
- - - % % % % % % - - - % % % % % % % % % 

384
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
%

- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - - - - x O O x x x x O O x - % % % - - 
% - - % % x O x - - - - x O O x % % - % % 
% - - % % x O x x x x - - x O O x % % % % 
- - % % % x O O O O O x x x x O x - % % % 
% % % - % - x x x x O O O O x O O x - % % 
% - - - % % - - % % x x x O x x O x % % - 
% % % % - % % - % % % - x O O O O x % % - 
- - - % - - - - % - % % % x x x x - - % % 
- - - % % % % % % - - - % % % % % % % % % 

400
- % % - - % % % % % % % % % % - - - - - - 
- % % % % % - x x x x x x - % - - - - - - 
% % % % - - x O O O O O O x % % % % % - - 
% - % % % x O O x x x x O O x - % % % - - 
% - % - % x O x - - - - x O O x % % - % % 
% - % % % x O x x x x - - x O O x % % % % 
- - % % % x O O O O O x x x x O x - % % % 
% % % - % - x x x x O O O O x O O x - % % 
% - - - % % - - % % x x x O x x O x % % - 
% % % % - % % - % % % - x O O O O x % % - 
- - - % - - - - % - % % % x x x x - - % % 
- - - 

KeyboardInterrupt: 

In [ ]:
check_udlr(grid, 9, 12)

In [ ]:
[x for x in adjacent(grid,9,12)]

In [ ]:
def score_adjacent(grid, x, y):
    initial_score = tile_score(grid,x,y)
    for i,j in adjacent(grid,x,y):
        initial_score += tile_score(grid, i, j)
    return initial_score

def grade_option(grid, x, y, direction):
    # grades placing a river and surrouding forests around it there
    # assert(grid[x][y],Tile.EMPTY, "Must be an empty tile")
    
    if direction == 'u':
        x -= 1
    elif direction == 'd':
        x += 1
    elif direction == 'r':
        y += 1
    elif direction == 'l':
        y -= 1
    else:
        raise ValueError("Direction must be u,d,l, or r")
    
    initial_score = score_adjacent(grid,x,y)
    
    prev_tile = grid[x][y]
    grid[x][y] = Tile.RIVER
    del_later = []
    for i,j in adjacent(grid,x,y):
        if grid[i][j] == Tile.EMPTY:
            grid[i][j] = Tile.THICKET
            del_later.append((i,j))
            
            
    final_score = score_adjacent(grid,x,y)
    
    grid[x][y] = prev_tile
    for i,j in del_later:
        grid[i][j] = Tile.EMPTY
        
    return final_score - initial_score
    
    
    
    
grade_option(grid, 9, 12, 'd')


In [ ]:
repc(grid)
score(grid)

In [ ]:
class Instance:
    def __init__(self, grid=just_road_fixture(), string='', start_coord=(5,0), last_coord=(5,0)):
        self.grid = grid
        self.string = string
        self.start_coord = start_coord
        self.last_coord = last_coord
        if last_coord is not None:
            x,y = last_coord
            self.grid[x][y] = Tile.RIVER
    
    def options(self):
        return check_udlr(self.grid, self.last_coord[0], self.last_coord[1])
    
#     def pick(self, x, y):
#         # assert(self.grid[x][y] == Tile.EMPTY)
#         self.grid[x][y] = Tile.RIVER
#         for i,j in adjacent(self.grid, x, y):
#             if self.grid[i][j] == Tile.EMPTY:
#                 self.grid[i][j] = Tile.THICKET
    
    def replace_around_rivers(self, target=Tile.EMPTY, replace=Tile.THICKET):
        x,y = self.start_coord
        for i,j in string_gen(self.grid, x, y, self.string):
            for ai,aj in adjacent(self.grid, i, j):
                if self.grid[ai][aj] == target:
                    self.grid[ai][aj] = replace
                    
    
    def score(self):
        # use self.string to generate Thickets
        x,y = self.start_coord
        self.replace_around_rivers()
        
        score = 0    
        for i,j in string_gen(self.grid, x, y, self.string):
            for ai,aj in adjacent(self.grid, i, j):
                score += tile_score(self.grid, ai, aj)
                
        self.replace_around_rivers(Tile.THICKET, Tile.EMPTY)
        return score
        

In [ ]:
# faster scoring 
# append all river adjacent tiles to a list
# add up all of those river scores

In [ ]:
repc(just_road_fixture())

In [ ]:
ins = Instance()

In [ ]:
ins.options()

In [ ]:
repc(ins.grid)

In [ ]:
ins.options()

In [ ]:
ins.grid = best_river_fixture()
repc(ins.grid)
ins.string = 'drdrddldlddrrururuuuululululuurrdrdrd'
score(ins.grid)

In [ ]:
class DFS:
    score = 0
    string = ''

def dfs(ss, string=''):
    score, options = ss.get_child(string)

    if score > 1000:
        return score

    if score > DFS.score:
        DFS.score = score
        DFS.string = string
        print(score, string)

    for ch in options:
        dfs(ss, string + ch)